In [1]:
import utils
import plotting
import numpy as np
import warnings
import pandas as pd
from pandas.core.common import SettingWithCopyWarning

# ignore these because they don't seem to apply
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

from PyPDF2 import PdfFileMerger

from copy import deepcopy

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
plot_dir = "../../../plots/ttsfs/Dec19"

import os

_ = os.system(f"mkdir -p {plot_dir}")

In [13]:
samples = {
    "QCD": "QCD",
    "Single Top": "ST",
    "TTSL": "TTToSemiLeptonic",
    "TTbar": ["TTTo2L2Nu", "TTToHadronic"],
    "W+Jets": "WJets",
    "Diboson": ["WW", "WZ", "ZZ"],
    "Data": "SingleMuon",
}

top_matched_key = "TT Top Matched"

data_dir = "../../../../data/ttsfs/Dec5_2/"
year = "2018"

# filters = [("('ak8FatJetPt', '0')", ">=", 500)]
filters = None

events_dict = utils.load_samples(data_dir, samples, year, filters=filters)

Finding QCD samples
Loading QCD_HT1000to1500_PSWeights_madgraph
Loaded 54 entries
Loading QCD_HT2000toInf_PSWeights_madgraph
Loaded 12 entries
No parquet file for QCD_HT50to100_PSWeights_madgraph
No parquet file for QCD_HT100to200_PSWeights_madgraph
Loading QCD_HT1500to2000_PSWeights_madgraph
Loaded 41 entries
Loading QCD_HT700to1000_PSWeights_madgraph
Loaded 154 entries
Loading QCD_HT200to300_PSWeights_madgraph
Loaded 0 entries
Loading QCD_HT300to500_PSWeights_madgraph
Loaded 0 entries
Loading QCD_HT500to700_PSWeights_madgraph
Loaded 74 entries
Finding Single Top samples
Loading ST_tW_antitop_5f_NoFullyHadronicDecays
Loaded 11639 entries
Loading ST_tW_top_5f_NoFullyHadronicDecays
Loaded 11547 entries
Loading ST_t-channel_top_4f_InclusiveDecays
Loaded 5096 entries
Loading ST_tW_top_5f_inclusiveDecays
Loaded 4504 entries
Loading ST_tW_antitop_5f_inclusiveDecays
Loaded 4313 entries
Loading ST_s-channel_4f_leptonDecays
Loaded 1493 entries
Loading ST_t-channel_top_5f_InclusiveDecays
Loaded

In [14]:
utils.get_cutflow(f"{data_dir}/{year}/TTToSemiLeptonic/pickles", year, "TTToSemiLeptonic")

{'top_has_bs': 67720.0,
 'ak8_jet': 222727.0,
 'ak4_jet': 67773.0,
 'muon': 2081716.0,
 'lepW': 166874.0,
 'all': 41977192,
 'npvsGood': 41977190.0,
 'met': 173100.0}

In [15]:
events_dict[top_matched_key] = events_dict["TTSL"].loc[events_dict["TTSL"]["top_matched"][0] == 1]
events_dict["TT W Matched"] = events_dict["TTSL"].loc[events_dict["TTSL"]["w_matched"][0] == 1]
events_dict["TT Unmatched"] = pd.concat(
    [events_dict["TTbar"], events_dict["TTSL"].loc[events_dict["TTSL"]["unmatched"][0] == 1]]
)
# del events_dict["TTSL"]
# del events_dict["TTbar"]

In [16]:
# normalize scale factors to average to 1
for key in ["lp_sf", "lp_sf_lnN"]:
    mean_lp_sfs = np.mean(events_dict[top_matched_key][key], axis=0)
    events_dict[top_matched_key].loc[:, key] = (
        events_dict[top_matched_key].loc[:, key] / mean_lp_sfs
    ).values

In [23]:
events_dict[top_matched_key]["lp_sf"]

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
3,0.760378,0.601804,0.807388,0.105286,0.327075,0.411378,0.186585,0.331885,0.321619,0.102886,...,0.528601,0.000000,0.371032,0.260599,0.568056,0.108599,0.663189,0.732413,0.776595,0.738116
17,0.068504,0.111595,0.136575,0.349294,0.025463,0.230662,0.052184,0.088294,0.009243,0.000000,...,0.061889,0.039558,0.034655,0.074090,0.079895,0.009616,0.000000,0.038720,0.000000,0.061386
51,0.011383,0.000000,0.000000,0.000000,0.000000,0.000000,0.194064,0.000000,0.000000,0.006171,...,0.000000,0.008171,0.005350,0.000000,0.000000,0.000000,1.449572,0.033778,0.000000,0.014286
60,0.598153,0.036358,0.088516,0.042792,0.000000,0.057375,0.016292,0.420628,0.200727,0.000000,...,0.227312,0.013483,0.047918,2.317237,0.046414,0.423753,0.628519,0.067978,0.081529,0.038612
62,0.416352,0.051930,0.030135,0.025318,0.153399,0.332428,0.219694,0.099034,0.276771,0.321003,...,0.421678,0.277841,0.438352,0.195278,0.533511,0.279500,0.378273,0.174890,0.253352,0.259057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65226,20.051270,0.000000,0.000000,8.944148,3.867852,20.753644,55.815220,0.000000,0.000000,5.875942,...,5.798396,2.295006,43.924229,0.000000,50.810962,131.999765,58.707277,0.000000,3.246527,9.170256
65431,0.000141,0.000000,0.000000,0.000000,0.000000,0.000000,0.000188,0.000000,0.000248,0.000000,...,0.000000,0.000797,0.000004,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000072
65460,0.022779,0.000000,0.021342,0.020385,0.097311,0.000000,0.001038,0.022776,0.000403,0.000459,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000049,0.000000,0.124392,0.000000
65474,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
np.logsp

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 12))
_ = plt.hist(events_dict[top_matched_key]["lp_sf"][0].values, np.logspace(-4, 2, 101, base=10), histtype='step')
plt.xscale("log")
# plt.yscale("log")
plt.xlabel("LP SF")
plt.title("Scale factor distribution")

In [8]:
# for key in events_dict:
#     if key == top_matched_key:
#         events_dict[key]["total_weight"] = events_dict[key]["weight"].values * events_dict[key]["lp_sf"]
#         events_dict[key]["total_weight_err"] = (
#             events_dict[key]["weight"] * events_dict[key]["lp_sf_err"]
#         )
#     else:
#         events_dict[key]["total_weight"] = events_dict[key]["weight"]
#         events_dict[key]["total_weight_err"] = events_dict[key]["weight"] * 0

In [17]:
for key in events_dict:
    events_dict[key] = events_dict[key][events_dict[key]["ak8FatJetPt"][0] >= 500]
    events_dict[key] = events_dict[key][events_dict[key]["ak8FatJetMsd"][0] >= 50]

In [18]:
plot_samples = [
    "QCD",
    "Diboson",
    "Single Top",
    "W+Jets",
    "TT Unmatched",
    "TT W Matched",
    top_matched_key,
]

### Pre plots

In [ ]:
# {var: (bins, label)}
plot_vars = {
    # "ak8FatJetMass": ([20, 125, 225], r"$m_{SD}$ (GeV)"),
    # "ak8FatJetPt": ([20, 500, 1200], r"$p_T$ (GeV)"),
    # "ak8FatJetnPFCands": ([20, 0, 120], r"# of PF Candidates"),
    "ak8FatJetParticleNet_Th4q": ([20, 0.8, 1], r"ParticleNet $T_{H4q}$ Non-MD"),
    # "tau21": ([20, 0.04, 0.8], r"$\tau_{21}$"),
    # "tau32": ([20, 0.2, 1], r"$\tau_{32}$"),
    # "tau43": ([20, 0.42, 1], r"$\tau_{43}$"),
}

pre_hists = {}

for var, (bins, label) in plot_vars.items():
    if var not in pre_hists:
        pre_hists[var] = utils.singleVarHistNoMask(
            events_dict, var, bins, label, weight_key="weight"
        )

merger_pre_plots = PdfFileMerger()

for var, var_hist in pre_hists.items():
    name = f"{plot_dir}/pre_{var}.pdf"
    plotting.ratioLinePlot(
        var_hist,
        plot_samples,
        year,
        bg_err=None,
        name=name,
    )
    merger_pre_plots.append(name)

# merger_pre_plots.write(f"{plot_dir}/PrePlots.pdf")
# merger_pre_plots.close()

### Post plots

In [ ]:
post_hists = {}
post_hists_err = {}

events = events_dict[top_matched_key]

for var, (bins, label) in plot_vars.items():
    if var not in post_hists:
        toy_hists = []
        for i in range(events["lp_sf"].shape[1]):
            toy_hists.append(
                np.histogram(
                    events[var][0].values.squeeze(),
                    np.linspace(*bins[1:], bins[0] + 1),
                    weights=events["weight"][0].values * events["lp_sf"][i].values,
                )[0]
            )

        nom_vals = toy_hists[0]  # first column are nominal values
        unc = np.minimum(nom_vals, np.std(toy_hists[1:], axis=0))  # cap at 100% unc

        thist = deepcopy(pre_hists[var])
        top_matched_key_index = np.where(np.array(list(thist.axes[0])) == top_matched_key)[0][0]
        thist.view(flow=False)[top_matched_key_index, :].value = nom_vals
        post_hists[var] = thist
        post_hists_err[var] = unc


merger_post_plots = PdfFileMerger()

for var, var_hist in post_hists.items():
    name = f"{plot_dir}/post_{var}.pdf"
    plotting.ratioLinePlot(
        var_hist,
        plot_samples,
        year,
        bg_err=post_hists_err[var],
        name=name,
    )
    merger_post_plots.append(name)

# merger_post_plots.write(f"{plot_dir}/PostPlots.pdf")
# merger_post_plots.close()

### Post LnN Plots

In [ ]:
post_hists = {}
post_hists_err = {}

events = events_dict[top_matched_key]

for var, (bins, label) in plot_vars.items():
    if var not in post_hists:
        toy_hists = []
        for i in range(events["lp_sf_lnN"].shape[1]):
            toy_hists.append(
                np.histogram(
                    events[var][0].values.squeeze(),
                    np.linspace(*bins[1:], bins[0] + 1),
                    weights=events["weight"][0].values * events["lp_sf_lnN"][i].values,
                )[0]
            )

        nom_vals = toy_hists[0]  # first column are nominal values
        unc = np.minimum(nom_vals, np.std(toy_hists, axis=0))  # cap at 100% unc

        thist = deepcopy(pre_hists[var])
        top_matched_key_index = np.where(np.array(list(thist.axes[0])) == top_matched_key)[0][0]
        thist.view(flow=False)[top_matched_key_index, :].value = nom_vals
        post_hists[var] = thist

        post_hists_err[var] = unc


merger_post_plots = PdfFileMerger()

for var, var_hist in post_hists.items():
    name = f"{plot_dir}/postlnN_{var}.pdf"
    plotting.ratioLinePlot(
        var_hist,
        plot_samples,
        year,
        bg_err=post_hists_err[var],
        name=name,
    )
    merger_post_plots.append(name)

# merger_post_plots.write(f"{plot_dir}/PostLnNPlots.pdf")
# merger_post_plots.close()

In [22]:
mass_hist = utils.singleVarHistNoMask(
    events_dict, "ak8FatJetMass", [20, 125, 225], r"$m_{SD}$", weight_key="weight"
)

In [ ]:
plotting.ratioHistPlot(
    mass_hist,
    ["QCD", "Diboson", "Single Top", "W+Jets", "TT Unmatched", "TT W Matched", top_matched_key],
    f"{plot_dir}/",
)